In [17]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import os
import librosa
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [18]:
# Function to extract features from audio files
def extract_features(audio_file_path):
    try:
        y, sr = librosa.load(audio_file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        # Resize the feature matrix to a fixed size (e.g., 40x173)
        resized_mfccs = np.resize(mfccs, (40, 173))  # You may adjust the size as needed
        return resized_mfccs
    except Exception as e:
        print(f"Error extracting features from {audio_file_path}: {e}")
        return None

In [19]:
# Initialize lists to store features and labels
X = []
y = []

In [20]:
# Process healthy data
healthy_data_path = 'C:/Users/rishi/Desktop/mini project-1/healthy'
for file_name in os.listdir(healthy_data_path):
    if file_name.endswith('.wav'):
        audio_file_path = os.path.join(healthy_data_path, file_name)
        features = extract_features(audio_file_path)
        if features is not None:
            X.append(features)
            y.append(0)  # Label for healthy patients

In [21]:
# Process unhealthy data
unhealthy_data_path = 'C:/Users/rishi/Desktop/mini project-1/unhealthy'
for file_name in os.listdir(unhealthy_data_path):
    if file_name.endswith('.wav'):
        audio_file_path = os.path.join(unhealthy_data_path, file_name)
        features = extract_features(audio_file_path)
        if features is not None:
            X.append(features)
            y.append(1)  # Label for unhealthy patients

In [22]:
# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [23]:
# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [24]:
# Reshape the input data for compatibility with LSTM layer
# For LSTM, the input shape should be (num_samples, timesteps, input_dim)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

In [32]:
from tensorflow import keras
# Define the RNN model architecture (replace with your desired architecture if needed)
model = keras.Sequential([
  keras.layers.LSTM(64, return_sequences=True, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
  keras.layers.LSTM(32, activation='relu'),
  keras.layers.Dense(16, activation='relu'),
  keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [33]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [34]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [35]:
# Train the model with validation data
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - accuracy: 0.5206 - loss: 146.8259 - val_accuracy: 0.5608 - val_loss: 327.6545
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.4386 - loss: 185.4194 - val_accuracy: 0.4797 - val_loss: 90.3103
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5009 - loss: 80.0843 - val_accuracy: 0.4662 - val_loss: 99.5671
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5000 - loss: 67.4738 - val_accuracy: 0.4662 - val_loss: 61.9652
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.5388 - loss: 118.5476 - val_accuracy: 0.5473 - val_loss: 187.9367
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.4518 - loss: 324.5692 - val_accuracy: 0.4730 - val_loss: 307.6949
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.4639 - loss: 229.8502 - val_accuracy: 0.4932 - val_loss: 37.1307
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.5368 - loss: 61.2114 - v

In [37]:
# Make predictions on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Thresholding for binary classification


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step


In [39]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5475 - loss: 37.8463
Test accuracy: 0.510869562625885


In [40]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')  # Macro averaging for imbalanced classes (optional)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Additionally, you can get a more detailed classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.5108695652173914
Precision: 0.4811320754716981
Recall: 0.5930232558139535
F1-score: 0.5099431818181819
              precision    recall  f1-score   support

           0       0.55      0.44      0.49        98
           1       0.48      0.59      0.53        86

    accuracy                           0.51       184
   macro avg       0.52      0.52      0.51       184
weighted avg       0.52      0.51      0.51       184

